In [1]:
import os
import pandas as pd

In [8]:
df = pd.read_csv(os.path.join(os.getcwd(), 'data', 'inputs', 'monthly_ret_etfs_macro_transf.csv'))

In [9]:
df

,date,SPY,XLI,XLE,XLK,XLV,XLU,XLF,XLY,XLP,...,CUSR0000SAC,CUSR0000SAD,CUSR0000SAS,CPIULFSL,CUSR0000SA0L2,CUSR0000SA0L5,PCEPI,DDURRG3M086SBEA,DNDGRG3M086SBEA,DSERRG3M086SBEA
0,2010-01-01,0.016923,0.013404,-0.000175,0.054673,0.009053,0.049903,0.008365,0.039395,-0.015370,...,0.004413,0.004091,-0.001738,0.000299,0.003212,0.000397,0.002184,-0.001284,0.003983,0.002112
1,2010-02-01,-0.033009,-0.014646,-0.028196,-0.084965,-0.005073,-0.058167,-0.004150,-0.029612,-0.012423,...,-0.003179,-0.001099,0.000527,-0.001151,-0.001333,-0.001305,-0.000031,-0.003551,-0.002218,0.001206
2,2010-03-01,0.019129,0.041935,-0.000176,0.005935,0.001263,-0.015442,0.031746,0.060846,0.032863,...,-0.001056,-0.001145,0.001430,0.000064,0.000645,0.000100,0.001415,-0.001435,-0.000044,0.002325
3,2010-04-01,0.062435,0.090139,0.044180,0.062632,0.028887,0.023554,0.097240,0.081884,0.034380,...,-0.001511,-0.002564,0.001267,0.000106,0.000250,0.000081,0.000544,-0.003841,-0.000730,0.001620
4,2010-06-01,-0.096974,-0.112625,-0.171444,-0.095607,-0.071302,-0.065117,-0.104880,-0.090794,-0.044270,...,-0.002530,-0.001953,0.000981,-0.000452,-0.000939,-0.000651,-0.000419,-0.003396,-0.003020,0.000875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,2022-04-01,0.034068,0.030304,0.112218,0.026279,0.052945,0.110821,-0.022764,0.050035,0.009701,...,0.000000,-0.000384,0.006619,0.003329,0.003480,0.003943,0.001993,0.001709,-0.003765,0.003986
105,2022-06-01,-0.018577,-0.026422,0.168738,-0.013195,-0.012050,0.021246,-0.011113,-0.094876,-0.067722,...,0.018105,0.004665,0.007818,0.012089,0.014690,0.012330,0.009861,0.007153,0.021173,0.006597
106,2022-07-01,-0.085476,-0.080340,-0.198652,-0.104607,-0.032919,-0.047829,-0.109524,-0.093679,-0.024020,...,-0.006158,0.003295,0.003632,-0.002069,-0.003218,-0.000754,-0.000932,-0.002165,-0.005561,0.000872
107,2022-08-01,0.055509,0.077391,0.041338,0.082228,0.013328,0.062137,0.036556,0.117837,0.037419,...,-0.004755,0.003005,0.006746,0.001421,0.000255,0.002013,0.002679,0.004809,-0.008099,0.005841
